In [26]:
import numpy as np
import pandas as pd


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        names = False
        # check whether it is pd.DataFrame
        lexicon = self.lexicon
        if isinstance(self.lexicon, pd.DataFrame):
            cols = self.lexicon.columns
            index = self.lexicon.index
            lexicon = self.lexicon.to_numpy()
            names = True
            
        literal = np.empty(lexicon.shape)
        for row in range(len(lexicon)):
            row_sum = sum(lexicon[row]*self.prior)
            row_itself = lexicon[row]
            for col in range(len(row_itself)):
                literal[row][col] = (row_itself[col]*self.prior[col]) / row_sum     
        if names:
            return pd.DataFrame(literal, columns = cols, index = index)
        return literal

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        sp = rownorm(self.speaker().T * self.prior)
        return sp


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'moustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())


Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,0.0
moustache,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
moustache,0.00,1.00,0.0


Здесь можно вывести импликатуру (**шляпа --> не усы**), так как здесь одно и то же выражение для разных ситуаций имеет разную вероятность. (То есть прагрматический слушатель с большей вероятностью считает, что объект будет описан словом "шляпа", если на нем будет надета только шляпа, и словом "усы", если у объекта будут усы (вне зависимости от наличия шляпы))

In [31]:
    msgs = ['hat', 'glasses', 'moustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod_2 = RSA(lexicon=lex, prior=[0.1,0.45, 0.45], costs=[0.0, 0.0, 0.0])
    print("\nPragmatic listener")
    display(basic_mod_2.listener())


Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,1.000000,0.0


Здесь возможность выразить скалярную импликатуру также остается, но она менее однозначна. Для r2 прагматический слушатель точно уверен, что говорящий опишет r2 через усы, но при этом он еще довольно уверен, что слушающий может описать его через шляпу, таким образом прагматическому слушателю будет сложнее сделать выбор.

# 1 игра
импликатуры:   
1) очки -> не шляпа 
2) усы ---> не очки  

In [65]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 0.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod_3 = RSA(lexicon=lex, prior=[1/3,1/3, 1/3], costs=[0.0, 0.0, 0.0])
print("\nPragmatic listener")
display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.000000,1.0,0.000000
glasses,0.600000,0.4,0.000000
moustache,0.333333,0.0,0.666667


# 2 игра
импликатуры:   
1) шляпа ---> не усы  
2) усы ---> не очки  
Рассмотрим на ее примере изменение параметров

In [53]:
    msgs = ['hat', 'glasses', 'moustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod_3 = RSA(lexicon=lex, prior=[1/3,1/3, 1/3], costs=[0.0, 0.0, 0.0])
    print("\nPragmatic listener")
    display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,0.600000,0.4


In [54]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod_3 = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])
print("\nPragmatic listener")
display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.263636,0.736364,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.532258,0.467742


При повышении исходной вероятности выбора объекта существует меньшая необходимость в формировании скалярной импликатуры (т.е. слушающий уже знает, что говорящий скорее всего имеет в виду этот объект). Если же исходная вероятность выбора объекта низкая, то слушающий скорее будет искать скалярную импликатуру, чтобы понять, что имел в виду говорящий (см. исчезновение 2 импликатуры и увеличение в разницы вероятностей импликатуры 1 (однако и первая имликатура становится менее четкой, как в случае выше)).


In [58]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod_3 = RSA(lexicon=lex, prior=[1/3,1/3, 1/3], costs=[-4.0, 0.0, 0.0])
print("\nPragmatic listener")
display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.982332,0.017668,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.746581,0.253419


In [60]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod_3 = RSA(lexicon=lex, prior=[1/3,1/3, 1/3], costs=[4.0, 0.0, 0.0])
print("\nPragmatic listener")
display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.504537,0.495463,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.051196,0.948804


При понижении цены высказывания разница между вероятностями этого высказывания (относительно объектов) увеличивается --> скалярная импликатура выводится лучше. При повышениии ситуация обратная.

In [57]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

#     print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod_3 = RSA(lexicon=lex, prior=[1/3,1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=4)
print("\nPragmatic listener")
display(basic_mod_3.listener())


Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.894737,0.105263


При увеличении альфа, одна из скалярных импликатур стала выводиться лучше. Вероятно, это происходит, так как говорящий рассчитывает, что слушающий вычислит импликатуру и выбирает то высказывание, опирающееся на импликатуры